## Topic-Mining mit BERTopic

In diesem Praktikum geht es darum, die Themen der **Debatten im Deutschen Bundestag** zu analysieren.

Die Debatten laden wir dabei über das [API des Dokumentations- und Informationssystems für Parlamentsmaterialien (DIP)](https://dip.bundestag.de/%C3%BCber-dip/hilfe/api) des Deutschen Bundestages.
Bitte beachten Sie die [Nutzungsbedingungen](https://dip.bundestag.de/%C3%BCber-dip/nutzungsbedingungen), falls Sie die Daten über das Praktikum hinaus nutzen möchten.

In [ ]:
%pip install deutschland[dip-bundestag] protobuf==3.20 bertopic pillow dotenv openai

In [ ]:
%load_ext dotenv

In [ ]:
%dotenv /home/archive/nlp/.env

In den Verzeichnissen `xml_crawler` bzw. `parser` befinden sich ein Crawler sowie ein Parser, um die Debatten im XML-Format herunterzuladen und weiterzuverarbeiten.

Alternativ können Sie weiter unten das "fertige" `DataFrame` (Stand 5. Juni 2025) als CSV-Datei laden.

In [ ]:
from xml_crawler.crawler import BTP_XML_Crawler
from parser.btp_parser import SpeechParser
import pandas as pd

In [ ]:
speeches = []
for u in BTP_XML_Crawler():
    with open(u, "r") as xml:
        data = SpeechParser.parse(xml.read())
        speeches += data

In [ ]:
df = pd.DataFrame(speeches)

In [ ]:
df['text'][0]

Für jeden Redebeitrag haben wir folgende Informationen:

| Feld | Beschreibung|
|------|-------------|
| **top_id** | Id des Tagesordnungspunkts |
| **speech_id** | Id des Redebeitrags. Enthält u.a. die Wahlperiode und die Sitzungsnummer |
| **speaker_id** | Id des/der Sprecher*in |
| **date** | Datum der Rede |
| **type** | Immer `rede` |
| **text** | Text des Redebeitrags |

In [ ]:
df

Für die weitere Verwendung speichern wir die Daten als CSV-Datei.

In [ ]:
df.to_parquet('debatten.parquet')

In [ ]:
import pandas as pd

df = pd.read_parquet('debatten.parquet')

### Aufgabe 1: Themenmodellierung mit BERTopic

- **Ziel**: Wenden Sie BERTopic auf den Datensatz der Debattenbeiträge an, um relevante Themen zu identifizieren.
- **Schritte**:
  1. Vorverarbeitung der Textdaten (ist schon durch den Parser erfolgt).
  2. Erstellen Sie ein BERTopic-Modell und trainieren Sie es mit den vorverarbeiteten Textdaten.
     Verwenden Sie dabei `bertopic.representation.OpenAI` als [*Representation Model*](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html).
  2. [Speichern Sie das Modell](https://maartengr.github.io/BERTopic/getting_started/serialization/serialization.html).

Wie viele Themen identifiziert Ihr Modell?

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import openai
from bertopic import BERTopic
from bertopic.representation import OpenAI

# Fine-tune topic representations with GPT
client = openai.OpenAI()
representation_model = OpenAI(client, model="gpt-4o-mini", chat=True)
topic_model = BERTopic(representation_model=representation_model)

In [ ]:
topics, probs = topic_model.fit_transform(df['text'])

### Aufgabe 2: Themenvisualisierung
- **Ziel:** Visualisieren Sie die identifizierten Themen mithilfe von BERTopic-Visualisierungstools.
- **Schritte:**
  1. Verwenden Sie die Funktion `get_topic_info()` von BERTopic, um Informationen zu den Themen zu erhalten.
  2. Erstellen Sie mit `visualize_topics()` eine "Landkarte" der Themen.
     Beachten Sie, dass Sie die Visualisierung als HTML speichern und anschließend mit `IPython.display.HTML` anzeigen müssen, da sonst die Anzeige in Jupyter nicht funktioniert.
     Schauen Sie sich die "Verortung" der Themen an: Passt diese aus Ihrer Sicht? Liegen ähnliche/verwandte Themen nah beieinander?
  4. Analysieren Sie die Häufigkeit der Themen.



In [ ]:
topic_model.get_topic_info()

In [ ]:
fig = topic_model.visualize_topics()

fig.write_html("my_visualization.html")

# Show the figure
from IPython.display import HTML
HTML(filename="my_visualization.html")

### Aufgabe 3: Zeitliche Entwicklung der Themen
- **Ziel:** Analysieren Sie, wie sich die Themen über die Zeit hinweg verändert haben.
- **Schritte:**
  1. Extrahieren Sie das Datum der Debattenbeiträge aus dem DataFrame.
  2. Erstellen Sie eine Zeitreihe, die darstellt, wie oft jedes Thema in verschiedenen Zeitintervallen (z.B. monatlich) auftaucht.
  3. Visualisieren Sie die Ergebnisse in einem Zeitdiagramm, wobei die Themen farblich hervorgehoben werden.

### Aufgabe 4: Weitere Analysen
- **Ziel:** Untersuchen Sie die Themen und die Debatten genauer, etwa anhand folgender Fragestellungen:
    - Gibt es Themen, an denen sich einzelne Parteien stärker oder weniger intensiv beteiligen?
    - Gibt es Debattenthemen, an denen sich männliche bzw. weibliche Abgeordnete häufiger zu Wort melden?  
- **Schritte:**
  Hier können Sie sich "austoben". Die Stammdaten der Abgeordneten finden Sie im Verzeichnis `stammdaten`.